In [17]:
from transformers import pipeline
import pandas as pd
import ast

model_name = "alsgyu/sentiment-analysis-fine-tuned-model"

analyzer = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name, device=-1)

Device set to use cpu


In [18]:
df=pd.read_csv("combine_topic_summary_cleaned.csv")
df.head()

,Park_nm,Topic,Count,Name,Representation,Representative_Docs
0,용산_효창공원,1,64,1_나무_김구_기념관_효창,"['나무', '김구', '기념관', '효창', '선생', '오래', '역사', '시...","['숲도 좋고 운동하기좋은곳 역사가 있고(윤봉길) 김구선생님 기념관도 있고', '도..."
1,용산_효창공원,2,43,2_역사_심속_의미_오래,"['역사', '심속', '의미', '오래', '여유', '휴식', '집기', '벌레...","['걷기와 휴식 역사가 있는 편안한 공원', '역사적 의미가 있는 공원', '여름철..."
2,용산_효창공원,4,16,4_방문_아름답_시내_최고,"['방문', '아름답', '시내', '최고', '한국', '조성', '괜찮', '추...",['아름답고 잘 관리된 공원 그런데 제가 서울에서 방문한 모든 공원은 완벽하게 관리...
3,종로_낙산공원,0,581,0_올라가_데이트_사람_예쁘,"['올라가', '데이트', '사람', '예쁘', '경치', '힘들', '주차', '...",['정상에 올라가면 나름 운치있고 성곽도 있고ㅎㅎ 예쁨^^ 사진찍기좋음ㅎㅎ 데이트 ...
4,종로_낙산공원,1,180,1_카페_도심_올라가_주차장,"['카페', '도심', '올라가', '주차장', '경치', '대학', '바라보', ...",['걸어서 성곽길을 걷다보면 아름다운 카페들도 나오고 운치있는곳들이 너무많은곳 연인...


In [19]:
def _join_docs(docs_field):
    if isinstance(docs_field, list):
        docs = docs_field
    else:
        docs = ast.literal_eval(docs_field)
    return " ".join([t for t in docs if isinstance(t, str) and t.strip()])

# 각 행의 Representative_Docs를 합쳐 문서 1개로 만듦
texts = df["Representative_Docs"].apply(_join_docs)

# 배치 추론(원형 유지: 같은 pipeline/analyzer 사용, truncation만 명시)
raw_results = analyzer(texts.tolist(), truncation=True)

# 라벨 매핑
label_mapping = {"LABEL_0": "부정", "LABEL_1": "중립", "LABEL_2": "긍정"}

# 최종 컬럼 추가
df["sentiment_label"] = [label_mapping[r["label"]] for r in raw_results]
df["sentiment_score"] = [float(r["score"]) for r in raw_results]  # 0~1 확률값

print(df[["sentiment_label", "sentiment_score"]].head(20))


   sentiment_label  sentiment_score
0               긍정         0.598557
1               긍정         0.971144
2               긍정         0.616219
3               긍정         0.824299
4               긍정         0.942347
5               긍정         0.814306
6               중립         0.644720
7               긍정         0.989085
8               긍정         0.986623
9               중립         0.699922
10              긍정         0.738534
11              긍정         0.979929
12              긍정         0.894467
13              긍정         0.980794
14              긍정         0.983154
15              중립         0.602471
16              긍정         0.959748
17              긍정         0.982432
18              긍정         0.974702
19              긍정         0.981843


In [20]:
output_file_path = "final_sentiment_analysis_results.csv"
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file_path}")

Results saved to final_sentiment_analysis_results.csv
